In [ ]:

import logging
import sys
import time
from threading import Event
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.positioning.motion_commander import MotionCommander
from cflib.utils import uri_helper


URI = uri_helper.uri_from_env(default='radio://0/80/2M/E7E7E7E7E7')


DEFAULT_HEIGHT = 1.0
BOX_LIMIT = 0.5
# deck_attached_event = Event()


def take_off_simple(scf):
    with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
        for i in range(10):
            mc.start_up(4)
            mc.stop()


cflib.crtp.init_drivers()
with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:

    # scf.cf.param.add_update_callback(group='deck', name='bcFlow2',
    #                                  cb=param_deck_flow)
    # time.sleep(1)

    # Arm the Crazyflie
    scf.cf.platform.send_arming_request(True)
    time.sleep(0.5)

    take_off_simple(scf)


In [ ]:
import logging
import time
from threading import Thread

import cflib
from cflib.crazyflie import Crazyflie
from cflib.utils import uri_helper


logging.basicConfig(level=logging.ERROR)

URI = uri_helper.uri_from_env(default='radio://0/80/2M/E7E7E7E7E8')

cflib.crtp.init_drivers(enable_debug_driver=False)

crazyflie = Crazyflie()
crazyflie.connected.add_callback(URI)
crazyflie.open_link(URI)
print('Connecting to %s' % URI)

roll    = 0
pitch   = 0
yawrate = 0
thrust  = 37000 
crazyflie.commander.send_setpoint(0, 0, 0, 0)

for i in range(1,30):
# take off
    if i < 10:
        crazyflie.commander.send_setpoint(roll, pitch, yawrate, thrust)
        time.sleep(0.1)
# increase height
    elif i > 10 and i < 15:
        thrust = 38000
        crazyflie.commander.send_setpoint(0, 0, yawrate, thrust)
        time.sleep(0.1)
# for hovering
    else:
        roll    = 0
        pitch   = 0
        thrust = 36000 # just about enough to balance weight
        crazyflie.commander.send_setpoint(roll, pitch, yawrate, thrust)
        time.sleep(0.1)            
    print ('Sending data..', thrust)
    
Fland = thrust
while Fland > 20000: 
    Fland += -500
    crazyflie.commander.send_setpoint(roll, pitch, yawrate, Fland)
    time.sleep(0.1)
    print (Fland)
crazyflie.close_link()